In [71]:
# В этом задании мы применим метод опорных векторов для определения того, 
# к какой из тематик относится новость: атеизм или космос.

# Загрузите объекты из новостного датасета 20 newsgroups
from sklearn import datasets
newsgroups = datasets.fetch_20newsgroups(subset='all', categories=['alt.atheism', 'sci.space'])
y = newsgroups.target
X = newsgroups.data

In [65]:
y[:5]

array([0, 0, 1, 1, 1], dtype=int32)

In [72]:
# Вычислите TF-IDF-признаки для всех текстов
# TF (term frequency) * IDF (inverse document frequency)
from sklearn.feature_extraction.text import TfidfVectorizer

v = TfidfVectorizer()
v.fit_transform({'hello, world','happy new year'})
print(v.get_feature_names())

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(X)
words = vectorizer.get_feature_names()
words[10000:10005]

['happy', 'hello', 'new', 'world', 'year']


['do', 'do1', 'doable', 'dobbs', 'dobyns']

In [7]:
# Подберите минимальный лучший параметр C
# Внимание, эта ячейка выполняется в SIEM_VM 6 минут
import numpy as np
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

# 10^6 не будет в сетке
grid = {'C': np.power(10.0, np.arange(-5, 6))}
cv = KFold(n_splits=5, shuffle=True, random_state=241)
clf = SVC(kernel='linear', random_state=241)
gs = GridSearchCV(clf, grid, scoring='accuracy', cv=cv)
gs.fit(X, y)

GridSearchCV(cv=KFold(n_splits=5, random_state=241, shuffle=True),
       error_score='raise',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=241, shrinking=True,
  tol=0.001, verbose=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'C': array([  1.00000e-05,   1.00000e-04,   1.00000e-03,   1.00000e-02,
         1.00000e-01,   1.00000e+00,   1.00000e+01,   1.00000e+02,
         1.00000e+03,   1.00000e+04,   1.00000e+05])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='accuracy', verbose=0)

In [37]:
index = 0
for paramC in gs.cv_results_['params']:
    print(paramC, 'score =', gs.cv_results_['mean_train_score'][index])
    index += 1

{'C': 1.0000000000000001e-05} score = 0.552631857502
{'C': 0.0001} score = 0.552631857502
{'C': 0.001} score = 0.552631857502
{'C': 0.01} score = 0.552631857502
{'C': 0.10000000000000001} score = 0.964305218238
{'C': 1.0} score = 0.999720083975
{'C': 10.0} score = 1.0
{'C': 100.0} score = 1.0
{'C': 1000.0} score = 1.0
{'C': 10000.0} score = 1.0
{'C': 100000.0} score = 1.0


In [10]:
minC = gs.best_estimator_.C
minC

1.0

In [38]:
# Обучите SVM по всей выборке с оптимальным параметром C, найденным на предыдущем шаге.
clf = SVC(kernel='linear', C=minC, random_state=241)
clf.fit(X, y)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=241, shrinking=True,
  tol=0.001, verbose=False)

In [40]:
# Найдите 10 слов с наибольшим абсолютным значением веса (веса хранятся в поле coef_ у svm.SVC). 
# Они являются ответом на это задание. 
# Укажите эти слова через запятую или пробел, в нижнем регистре, в лексикографическом порядке.
clf.coef_

<1x28382 sparse matrix of type '<class 'numpy.float64'>'
	with 18404 stored elements in Compressed Sparse Row format>

In [50]:
import pandas as pd
coef = pd.DataFrame(clf.coef_.data, clf.coef_.indices)
coef[:10]

,0
11098,0.113315
6775,0.051343
5107,0.054452
98,0.059766
27042,0.104719
22622,0.104719
6135,0.104719
27130,0.006843
27083,0.006843
26026,0.007219


In [75]:
top_words = coef[0].map(lambda w: abs(w)).sort_values(ascending=False).head(10).index.map(lambda i: words[i])
top_words.sort_values(ascending=True)

Index(['atheism', 'atheists', 'bible', 'god', 'keith', 'moon', 'religion',
       'sci', 'sky', 'space'],
      dtype='object')

In [82]:
print (','.join(map(lambda x: x, top_words.values)))

space,god,atheism,atheists,moon,sky,religion,bible,keith,sci
